# Загрузка данных

In [786]:
import pandas as pd
import json

In [787]:
# FILE_NAME='data10-04.xlsx'
FILE_NAME='data.xlsx'

In [788]:
df = pd.read_excel(f'datasets/{FILE_NAME}')

In [789]:
df.head()

,F_User,C_Record_ID,C_Table,N_Operation,C_Json,D_Date_Created
0,64b18dc9-69d0-4286-887f-7469ebc1b47d,9770f581-b89f-4fb3-94f5-93173bb85ad1,ED_Meter_Readings-ListView,1,"{""LINK"":""50fb47a1-6711-4e65-9232-3700daaa58bd""...",2023-04-09 16:42:47.637
1,64b18dc9-69d0-4286-887f-7469ebc1b47d,5f97a8be-b3ad-4982-9191-fbed301b3c69,DD_Doc_Details-ListView,1,"{""LINK"":""a95caaa0-a674-4f31-84d4-cbb52f0317f1""...",2023-04-09 16:42:47.980
2,64b18dc9-69d0-4286-887f-7469ebc1b47d,d4c614dd-3114-4326-8ddd-bb9d74382b03,DD_Docs_Out-ListView,0,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 16:42:48.627
3,64b18dc9-69d0-4286-887f-7469ebc1b47d,9ee02cde-4a6f-4c51-8e3b-b14ddae38bee,DD_Files-ListView,0,"{""LINK"":""e010d6d0-1df5-790d-19fe-b007d7be4207""...",2023-04-09 16:42:49.557
4,64b18dc9-69d0-4286-887f-7469ebc1b47d,6829a2cd-007c-45d8-88ca-e143617555c4,DD_Docs-ListView,1,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 16:42:50.010


In [790]:
df['D_Date_Created'] = pd.to_datetime(df['D_Date_Created'])
df['N_Operation'] = df['N_Operation'].astype('int8')

In [791]:
df.drop_duplicates(inplace=True)

In [792]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88288 entries, 0 to 96645
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   F_User          88288 non-null  object        
 1   C_Record_ID     88288 non-null  object        
 2   C_Table         88288 non-null  object        
 3   N_Operation     88288 non-null  int8          
 4   C_Json          87881 non-null  object        
 5   D_Date_Created  88288 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int8(1), object(4)
memory usage: 4.1+ MB


# Алгоритмы

In [793]:
users = df['F_User'].unique()

In [794]:
use_records = []
correct_records = []

## Алгоритм

Последовательность ED_Meter_Readings-ListView -> DD_Doc_Details-ListView -> DD_Docs_Out-ListView -> DD_Files-ListView -> DD_Docs-ListView

In [795]:
dict_algoritm_1 = {}
for i in range(len(users)):
    dict_algoritm_1[users[i]] = []

In [796]:
failed1 = []
def algoritm_1(user_id):    
    items = df.loc[df['F_User'] == user_id].sort_values(by=['D_Date_Created'], ascending=True)

    indexes = list(items.index)
    tables = list(items['C_Table'])
    operations = list(items['N_Operation'])
    json_items = list(items['C_Json'])
    resords = list(items['C_Record_ID'])

    buffer = {
        'ED_Meter_Readings-ListView': [],
        'DD_Doc_Details-ListView': [],
        'DD_Docs_Out-ListView': [],
        'DD_Files-ListView': [],
        'DD_Docs-ListView': [],
        'Other': []
    }
    
    for i in range(len(tables)):
        table = tables[i]
        operation = operations[i]
        json_item = json_items[i]
        index = indexes[i]
        record = resords[i]

        try:
            if table == 'ED_Meter_Readings-ListView':
                buffer['ED_Meter_Readings-ListView'].append([index, record, table, operation, json.loads(json_item)])
                
            if table == 'DD_Doc_Details-ListView':
                buffer['DD_Doc_Details-ListView'].append([index, record, table, operation, json.loads(json_item)])
                
            if table == 'DD_Docs_Out-ListView':
                buffer['DD_Docs_Out-ListView'].append([index, record, table, operation, json.loads(json_item)])
                
            if table == 'DD_Files-ListView':
                try:
                    buffer['DD_Files-ListView'].append([index, record, table, operation, json.loads(json_item)])
                except:
                    buffer['DD_Files-ListView'].append([index, record, table, operation, None])
                
            if table == 'DD_Docs-ListView':
                buffer['DD_Docs-ListView'].append([index, record, table, operation, json.loads(json_item)])
                
                if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
                    for key in buffer:
                        for j in range(len(buffer[key])):
                            if buffer['DD_Docs_Out-ListView'][0][4]['LINK'] == buffer['DD_Docs-ListView'][0][4]['LINK']:
                                dict_algoritm_1[user_id].append(buffer[key][j])
                            else:
                                correct_records.append(buffer[key][j][1])
                                
                            use_records.append(buffer[key][j][1])
                            
                            df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]
                            
                buffer = {
                    'ED_Meter_Readings-ListView': [],
                    'DD_Doc_Details-ListView': [],
                    'DD_Docs_Out-ListView': [],
                    'DD_Files-ListView': [],
                    'DD_Docs-ListView': [],
                    'Other': []
                }
        except json.JSONDecodeError:
            failed1.append(items.loc[index])
            
        if table not in (['ED_Meter_Readings-ListView', 'DD_Doc_Details-ListView', 'DD_Docs_Out-ListView', 'DD_Files-ListView', 'DD_Docs-ListView']):
            try:
                buffer['Other'].append([index, record, table, operation, json.loads(json_item)])
            except:
                buffer['Other'].append([index, record, table, operation, None])
            
    # дозаписываем данные
    if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
        for key in buffer:
            for j in range(len(buffer[key])):
                dict_algoritm_1[user_id].append(buffer[key][j])
                use_records.append(buffer[key][j][1])
                
                df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]

In [797]:
# use_records = []
# correct_records = []
# dict_algoritm_1 = {'7d865ee6-9803-427f-81f4-c8f956f6b46b': []}
# algoritm_1('7d865ee6-9803-427f-81f4-c8f956f6b46b')

# df['Use'] = df['C_Record_ID'].isin(use_records)

# display(df.loc[df['F_User'] == '7d865ee6-9803-427f-81f4-c8f956f6b46b', ['C_Record_ID', 'C_Table', 'D_Date_Created', 'Group_ID', 'Use']].sort_values(by=['D_Date_Created'], ascending=True).head(50))

In [798]:
for i in range(len(users)):
    user_id = users[i]
    algoritm_1(user_id)

In [799]:
failed_df1 = pd.DataFrame(failed1)
df = df.drop(index=failed_df1.index)

In [844]:
failed_df

,F_User,C_Record_ID,C_Table,N_Operation,C_Json,D_Date_Created,Group_ID
39772,7d865ee6-9803-427f-81f4-c8f956f6b46b,3802fd39-cd9a-4105-b480-37851dd8bb4b,DD_Doc_Details-ListView,1,"{""LINK"":""06f7ddf5-35ef-4a82-bb1a-8bd0e8e02259""...",2023-04-11 16:31:18.040,NaN
31962,c591cc56-d279-457e-80e6-4208b4f69adf,7767f2de-07be-4f28-9c72-8b2a939e5ada,DD_Docs-ListView,1,"{""LINK"":""0877f1f7-d19f-4331-abc7-ce137054581f""...",2023-04-11 09:45:13.360,NaN
60642,c591cc56-d279-457e-80e6-4208b4f69adf,f8ab2155-5ad9-49bf-b88d-b598bef48dae,DD_Docs_Out-ListView,1,"{""LINK"":""419baa7a-4a97-de93-c388-e0ef3d8b29da""...",2023-04-11 09:45:51.457,NaN
685,ebfbf941-f890-4d93-9e0b-15385f16a0ea,aa25717b-5067-4dee-8292-47e86021d5cc,DD_Docs-ListView,1,"{""LINK"":""2de685dc-81fb-489c-808e-497bcfacba86""...",2023-04-09 20:16:39.093,NaN
81678,ed0a59de-68f7-4f1c-8675-c01880e51d21,d59a7ff8-2163-45a4-90de-bdf22f27a369,DD_Docs_Out-ListView,0,"{""LINK"":""6785c523-1eb7-9c15-ed93-3376d3f69d32""...",2023-04-12 15:07:17.597,NaN
...,...,...,...,...,...,...,...
95178,3bad1236-f6a3-4bd9-ae4a-f90fdbb82ae7,0479e4ce-2854-4feb-975c-79d371717c50,DD_Docs_Out-ListView,0,"{""LINK"":""021033d2-4ac6-f445-01f6-53ccbc209706""...",2023-04-12 15:55:52.853,NaN
89994,3bad1236-f6a3-4bd9-ae4a-f90fdbb82ae7,215764cc-66e2-4b86-aecc-4ded92b9b88d,DD_Docs_Out-ListView,0,"{""LINK"":""45538abf-46bb-0e15-f67f-d3cab7c36bb7""...",2023-04-12 16:52:56.467,NaN
69147,8476b258-fb41-4223-b857-20cbe663eda9,675f4f06-1add-4834-b1ac-690553c1ecaf,DD_Docs_Out-ListView,0,"{""LINK"":""cf350d26-42c5-eebb-86a1-43552a2316a3""...",2023-04-12 12:02:59.077,NaN
71119,8476b258-fb41-4223-b857-20cbe663eda9,164073e6-dfbf-469a-ae91-9d9eafffa59e,DD_Docs_Out-ListView,0,"{""LINK"":""1a2eadf1-5908-7044-477c-cd0891a0d8a6""...",2023-04-12 12:21:11.223,NaN


In [843]:
str(failed_df.loc[31962, 'C_Json'])

'{"LINK":"0877f1f7-d19f-4331-abc7-ce137054581f","F_Statuses":{"LINK":1,"C_Name":"В работе","C_Const":"DS_IN_WORK","N_Code":0,"S_Create_Date":null,"S_Modif_Date":null,"S_Creator":"","S_Owner":""},"F_Types":{"LINK":20,"C_Name":"Маршрутный лист по УП"},"D_Date":"2023-04-03T00:00:00.000Z","D_Date_End":"2023-04-30T23:59:00.000Z","C_Number":"ССП_Запрудня_2","F_Users":{"LINK":"127c5e56-d17e-42b3-8288-1648382c4403","C_Fio":"Поликар'

In [800]:
df['Use'] = df['C_Record_ID'].isin(use_records)
print(f'Обработано записей: {len(use_records)}')

Обработано записей: 74689


In [801]:
df.to_excel('outputs/algoritm_1.xlsx')

## Алгоритм

Последовательность ED_Meter_Readings-ListView -> DD_Docs_Out-ListView -> DD_Files-ListView -> DD_Docs-ListView

In [802]:
# dict_algoritm_2 = {}
# for i in range(len(users)):
#     dict_algoritm_2[users[i]] = []

In [803]:
# def algoritm_2(user_id):    
#     items = df.loc[(df['F_User'] == user_id) & (df['Use'] == False)].sort_values(by=['D_Date_Created'], ascending=True)

#     indexes = list(items.index)
#     tables = list(items['C_Table'])
#     operations = list(items['N_Operation'])
#     json_items = list(items['C_Json'])
#     resords = list(items['C_Record_ID'])

#     buffer = {
#         'ED_Meter_Readings-ListView': [],
#         'DD_Docs_Out-ListView': [],
#         'DD_Files-ListView': [],
#         'DD_Docs-ListView': [],
#         'Other': []
#     }
    
#     is_meter = False
#     is_not_meter = False
    
#     for i in range(len(tables)):
#         table = tables[i]
#         operation = operations[i]
#         json_item = json_items[i]
#         index = indexes[i]
#         record = resords[i]

#         if table == 'ED_Meter_Readings-ListView':
#             is_meter = True
            
#             if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
                 
#                 for key in buffer:
#                     for j in range(len(buffer[key])):
#                         if buffer['DD_Docs_Out-ListView'][0][4]['LINK'] == buffer['DD_Docs-ListView'][0][4]['LINK']:
#                             dict_algoritm_2[user_id].append(buffer[key][j])
#                         else:
#                             correct_records.append(buffer[key][j][1])
                            
#                         use_records.append(buffer[key][j][1])
                            
#             if is_not_meter == True:   
#                 buffer = {
#                     'ED_Meter_Readings-ListView': [],
#                     'DD_Docs_Out-ListView': [],
#                     'DD_Files-ListView': [],
#                     'DD_Docs-ListView': [],
#                     'Other': []
#                 }
                
#             buffer['ED_Meter_Readings-ListView'].append([index, record, table, operation, json.loads(json_item)])
                
#             is_not_meter = False
#         else:
#             is_meter = False
            
#         if table == 'DD_Docs_Out-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0:
#             buffer['DD_Docs_Out-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
#         if table == 'DD_Files-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0:
#             try:
#                 buffer['DD_Files-ListView'].append([index, record, table, operation, json.loads(json_item)])
#             except:
#                 buffer['DD_Files-ListView'].append([index, record, table, operation, None])
            
#         if table == 'DD_Docs-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0:
#             buffer['DD_Docs-ListView'].append([index, record, table, operation, json.loads(json_item)])
#             is_not_meter = True
            
#         if table not in (['ED_Meter_Readings-ListView', 'DD_Docs_Out-ListView', 'DD_Files-ListView', 'DD_Docs-ListView']):
#             try:
#                 buffer['Other'].append([index, record, table, operation, json.loads(json_item)])
#             except:
#                 buffer['Other'].append([index, record, table, operation, None])
            
#     # дозаписываем данные
#     if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
#         for key in buffer:
#             for j in range(len(buffer[key])):
#                 dict_algoritm_2[user_id].append(buffer[key][j])
#                 use_records.append(buffer[key][j][1])

In [804]:
# for i in range(len(users)):
#     user_id = users[i]
#     algoritm_2(user_id)

In [805]:
# df['Use'] = df['C_Record_ID'].isin(use_records)
# print(f'Обработано записей: {len(use_records)}')

In [806]:
# df.to_excel('outputs/algoritm_2.xlsx')

## Алгоритм

Последовательность ED_Meter_Readings-ListView -> DD_Doc_Details-ListView -> DD_Docs_Out-ListView -> DD_Docs-ListView

In [807]:
# dict_algoritm_3 = {}
# for i in range(len(users)):
#     dict_algoritm_3[users[i]] = []

In [808]:
# def algoritm_3(user_id):    
#     items = df.loc[(df['F_User'] == user_id) & (df['Use'] == False)].sort_values(by=['D_Date_Created'], ascending=True)

#     indexes = list(items.index)
#     tables = list(items['C_Table'])
#     operations = list(items['N_Operation'])
#     json_items = list(items['C_Json'])
#     resords = list(items['C_Record_ID'])

#     buffer = {
#         'ED_Meter_Readings-ListView': [],
#         'DD_Doc_Details-ListView': [],
#         'DD_Docs_Out-ListView': [],
#         'DD_Docs-ListView': [],
#         'Other': []
#     }
    
#     is_meter = False
#     is_not_meter = False
    
#     for i in range(len(tables)):
#         table = tables[i]
#         operation = operations[i]
#         json_item = json_items[i]
#         index = indexes[i]
#         record = resords[i]

#         if table == 'ED_Meter_Readings-ListView':
#             is_meter = True
            
#             if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Doc_Details-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
                
#                 for key in buffer:
#                     for j in range(len(buffer[key])):
#                         if buffer['DD_Docs_Out-ListView'][0][4]['LINK'] == buffer['DD_Docs-ListView'][0][4]['LINK']:
#                             dict_algoritm_3[user_id].append(buffer[key][j])
#                         else:
#                             correct_records.append(buffer[key][j][1])
                            
#                         use_records.append(buffer[key][j][1])
                            
#             if is_not_meter == True:   
#                 buffer = {
#                     'ED_Meter_Readings-ListView': [],
#                     'DD_Doc_Details-ListView': [],
#                     'DD_Docs_Out-ListView': [],
#                     'DD_Docs-ListView': [],
#                     'Other': []
#                 }
                
#             buffer['ED_Meter_Readings-ListView'].append([index, record, table, operation, json.loads(json_item)])
                
#             is_not_meter = False
#         else:
#             is_meter = False
            
#         if table == 'DD_Doc_Details-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0:
#             buffer['DD_Doc_Details-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
#         if table == 'DD_Docs_Out-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Doc_Details-ListView']) > 0:
#             buffer['DD_Docs_Out-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
#         if table == 'DD_Docs-ListView' and len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Doc_Details-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0:
#             buffer['DD_Docs-ListView'].append([index, record, table, operation, json.loads(json_item)])
#             is_not_meter = True
        
#         if table not in (['ED_Meter_Readings-ListView', 'DD_Doc_Details-ListView', 'DD_Docs_Out-ListView', 'DD_Docs-ListView']):
#             try:
#                 buffer['Other'].append([index, record, table, operation, json.loads(json_item)])
#             except:
#                 buffer['Other'].append([index, record, table, operation, None])
            
#     # дозаписываем данные
#     if len(buffer['ED_Meter_Readings-ListView']) > 0 and len(buffer['DD_Doc_Details-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
#         for key in buffer:
#             for j in range(len(buffer[key])):
#                 dict_algoritm_3[user_id].append(buffer[key][j])
#                 use_records.append(buffer[key][j][1])

In [809]:
# for i in range(len(users)):
#     user_id = users[i]
#     algoritm_3(user_id)

In [810]:
# df['Use'] = df['C_Record_ID'].isin(use_records)
# print(f'Обработано записей: {len(use_records)}')

In [811]:
# df.to_excel('outputs/algoritm_3.xlsx')

## Алгоритм

Последовательность DD_Doc_Details-ListView -> DD_Docs_Out-ListView -> DD_Docs-ListView

In [812]:
dict_algoritm_4 = {}
for i in range(len(users)):
    dict_algoritm_4[users[i]] = []

In [813]:
def algoritm_4(user_id):    
    items = df.loc[(df['F_User'] == user_id) & (df['Use'] == False)].sort_values(by=['D_Date_Created'], ascending=True)

    indexes = list(items.index)
    tables = list(items['C_Table'])
    operations = list(items['N_Operation'])
    json_items = list(items['C_Json'])
    resords = list(items['C_Record_ID'])

    buffer = {
        'DD_Doc_Details-ListView': [],
        'DD_Docs_Out-ListView': [],
        'DD_Docs-ListView': [],
        'Other': []
    }
    
    for i in range(len(tables)):
        table = tables[i]
        operation = operations[i]
        json_item = json_items[i]
        index = indexes[i]
        record = resords[i]

        if table == 'DD_Doc_Details-ListView': 
            buffer['DD_Doc_Details-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
        if table == 'DD_Docs_Out-ListView':
            buffer['DD_Docs_Out-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
        if table == 'DD_Docs-ListView':
            buffer['DD_Docs-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
            if len(buffer['DD_Doc_Details-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
                for key in buffer:
                    for j in range(len(buffer[key])):
                        if buffer['DD_Docs_Out-ListView'][0][4]['LINK'] == buffer['DD_Docs-ListView'][0][4]['LINK']:
                            dict_algoritm_4[user_id].append(buffer[key][j])
                        else:
                            correct_records.append(buffer[key][j][1])

                        use_records.append(buffer[key][j][1])

                        df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]
                            
            buffer = {
                'DD_Doc_Details-ListView': [],
                'DD_Docs_Out-ListView': [],
                'DD_Docs-ListView': [],
                'Other': []
            }

        if table not in (['DD_Doc_Details-ListView', 'DD_Docs_Out-ListView', 'DD_Docs-ListView']):
            try:
                buffer['Other'].append([index, record, table, operation, json.loads(json_item)])
            except:
                buffer['Other'].append([index, record, table, operation, None])
            
    # дозаписываем данные
    if len(buffer['DD_Doc_Details-ListView']) > 0 and len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
        for key in buffer:
            for j in range(len(buffer[key])):
                dict_algoritm_4[user_id].append(buffer[key][j])
                use_records.append(buffer[key][j][1])
                
                df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]

In [814]:
for i in range(len(users)):
    user_id = users[i]
    algoritm_4(user_id)

In [815]:
df['Use'] = df['C_Record_ID'].isin(use_records)
print(f'Обработано записей: {len(use_records)}')

Обработано записей: 82513


In [816]:
df.to_excel('outputs/algoritm_4.xlsx')

## Алгоритм

Последовательность DD_Docs_Out-ListView -> DD_Files-ListView -> DD_Docs-ListView

In [817]:
dict_algoritm_5 = {}
for i in range(len(users)):
    dict_algoritm_5[users[i]] = []

In [818]:
def algoritm_5(user_id):    
    items = df.loc[(df['F_User'] == user_id) & (df['Use'] == False)].sort_values(by=['D_Date_Created'], ascending=True)

    indexes = list(items.index)
    tables = list(items['C_Table'])
    operations = list(items['N_Operation'])
    json_items = list(items['C_Json'])
    resords = list(items['C_Record_ID'])

    buffer = {
        'DD_Docs_Out-ListView': [],
        'DD_Files-ListView': [],
        'DD_Docs-ListView': [],
        'Other': []
    }
    
    for i in range(len(tables)):
        table = tables[i]
        operation = operations[i]
        json_item = json_items[i]
        index = indexes[i]
        record = resords[i]

        if table == 'DD_Docs_Out-ListView':
            buffer['DD_Docs_Out-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
        if table == 'DD_Files-ListView':
            try:
                buffer['DD_Files-ListView'].append([index, record, table, operation, json.loads(json_item)])
            except:
                buffer['DD_Files-ListView'].append([index, record, table, operation, None])
            
        if table == 'DD_Docs-ListView':
            buffer['DD_Docs-ListView'].append([index, record, table, operation, json.loads(json_item)])
            
            if len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
                for key in buffer:
                    for j in range(len(buffer[key])):
                        if buffer['DD_Docs_Out-ListView'][0][4]['LINK'] == buffer['DD_Docs-ListView'][0][4]['LINK']:
                            dict_algoritm_5[user_id].append(buffer[key][j])
                        else:
                            correct_records.append(buffer[key][j][1])
                            
                        use_records.append(buffer[key][j][1])
                        
                        df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]
            buffer = {
                'DD_Docs_Out-ListView': [],
                'DD_Files-ListView': [],
                'DD_Docs-ListView': [],
                'Other': []
            }
        
        if table not in (['DD_Docs_Out-ListView', 'DD_Files-ListView', 'DD_Docs-ListView']):
            try:
                buffer['Other'].append([index, record, table, operation, json.loads(json_item)])
            except:
                buffer['Other'].append([index, record, table, operation, None])
            
    # дозаписываем данные
    if len(buffer['DD_Docs_Out-ListView']) > 0 and len(buffer['DD_Files-ListView']) > 0 and len(buffer['DD_Docs-ListView']) > 0:
        for key in buffer:
            for j in range(len(buffer[key])):
                dict_algoritm_5[user_id].append(buffer[key][j])
                use_records.append(buffer[key][j][1])
                
                df.loc[df.index == buffer[key][j][0], 'Group_ID'] = buffer['DD_Docs_Out-ListView'][0][1]

In [819]:
for i in range(len(users)):
    user_id = users[i]
    algoritm_5(user_id)

In [820]:
df['Use'] = df['C_Record_ID'].isin(use_records)
print(f'Обработано записей: {len(use_records)}')

Обработано записей: 84113


In [821]:
df.to_excel('outputs/algoritm_5.xlsx')

In [822]:
df['Group_ID'].nunique()

12691

In [823]:
df['F_Doc_Details'] = ''

In [824]:
df['C_Table'].isin(['ED_Meter_Readings-ListView', 'DD_Doc_Details-ListView']).sum()

27130

## Заполнение DD_Doc_Details

In [825]:
df['Fixed'] = df['Use']
df['F_Doc_Details'] = ''
groups = list(df['Group_ID'].unique())
for groups_id in groups:
    data = df[(df['Group_ID'] == groups_id) & (record['Use'] == True)]

    detail_links = []

    meters_index = data['C_Table'].isin(['ED_Meter_Readings-ListView'])
    meters_data = data[meters_index].apply(lambda x: json.loads(x['C_Json']), axis=1)
    if len(meters_data) > 0:
        for i in meters_data:
            if bool(i['F_Doc_Details']):
                detail_links.append(i['F_Doc_Details']['LINK'])

    
    details_index = data['C_Table'].isin(['DD_Doc_Details-ListView'])
    details_data = data[details_index].apply(lambda x: json.loads(x['C_Json']), axis=1)
    if len(details_data) > 0:
        for i in details_data:
            detail_links.append(i['LINK'])

    found_detail = list(set(detail_links))

    if len(found_detail) > 1:
        df.loc[data.index, 'F_Doc_Details'] = json.dumps(found_detail)
        df.loc[data.index, 'Fixed'] = False

    if len(found_detail) == 1:
        df.loc[data.index, 'F_Doc_Details'] = found_detail[0]
        df.loc[data.index, 'Fixed'] = True

    if len(found_detail) == 0:
        df.loc[data.index, 'F_Doc_Details'] = ''


In [826]:
# df.to_excel('outputs/F_Doc_Details.xlsx')

## Объединяем группы DD_Docs_Out
Сперва все у кого DD_Doc_Details одинаковый берем как один акт а потом если у них операция = 0 то обратно оставляем Group_ID, отдельные группы

In [827]:
fixed_groups = df[(df['Fixed'] == True) & (df['F_Doc_Details'] != '')][['Group_ID', 'F_Doc_Details']].groupby(['F_Doc_Details']).agg({'Group_ID': 'first'}).reset_index().rename(columns={'Group_ID': 'Group_ID_Fixed'})

In [828]:
df = df.merge(fixed_groups, left_on='F_Doc_Details', right_on='F_Doc_Details')
df[df['F_Doc_Details'] == 'a95caaa0-a674-4f31-84d4-cbb52f0317f1']

,F_User,C_Record_ID,C_Table,N_Operation,C_Json,D_Date_Created,Group_ID,Use,F_Doc_Details,Fixed,Group_ID_Fixed
0,64b18dc9-69d0-4286-887f-7469ebc1b47d,9770f581-b89f-4fb3-94f5-93173bb85ad1,ED_Meter_Readings-ListView,1,"{""LINK"":""50fb47a1-6711-4e65-9232-3700daaa58bd""...",2023-04-09 16:42:47.637,d4c614dd-3114-4326-8ddd-bb9d74382b03,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
1,64b18dc9-69d0-4286-887f-7469ebc1b47d,5f97a8be-b3ad-4982-9191-fbed301b3c69,DD_Doc_Details-ListView,1,"{""LINK"":""a95caaa0-a674-4f31-84d4-cbb52f0317f1""...",2023-04-09 16:42:47.980,d4c614dd-3114-4326-8ddd-bb9d74382b03,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
2,64b18dc9-69d0-4286-887f-7469ebc1b47d,d4c614dd-3114-4326-8ddd-bb9d74382b03,DD_Docs_Out-ListView,0,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 16:42:48.627,d4c614dd-3114-4326-8ddd-bb9d74382b03,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
3,64b18dc9-69d0-4286-887f-7469ebc1b47d,9ee02cde-4a6f-4c51-8e3b-b14ddae38bee,DD_Files-ListView,0,"{""LINK"":""e010d6d0-1df5-790d-19fe-b007d7be4207""...",2023-04-09 16:42:49.557,d4c614dd-3114-4326-8ddd-bb9d74382b03,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
4,64b18dc9-69d0-4286-887f-7469ebc1b47d,6829a2cd-007c-45d8-88ca-e143617555c4,DD_Docs-ListView,1,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 16:42:50.010,d4c614dd-3114-4326-8ddd-bb9d74382b03,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
5,64b18dc9-69d0-4286-887f-7469ebc1b47d,9c968f25-e25e-4082-b15d-cdb26fa88aee,DD_Doc_Details-ListView,1,"{""LINK"":""a95caaa0-a674-4f31-84d4-cbb52f0317f1""...",2023-04-09 16:57:15.560,a0ec21d9-9722-4e23-9b03-b36ca2f1f252,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
6,64b18dc9-69d0-4286-887f-7469ebc1b47d,a0ec21d9-9722-4e23-9b03-b36ca2f1f252,DD_Docs_Out-ListView,1,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 16:57:16.460,a0ec21d9-9722-4e23-9b03-b36ca2f1f252,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
7,64b18dc9-69d0-4286-887f-7469ebc1b47d,59e00450-e9f8-45a3-aacb-e95521632320,DD_Docs-ListView,1,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 16:57:16.960,a0ec21d9-9722-4e23-9b03-b36ca2f1f252,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
8,64b18dc9-69d0-4286-887f-7469ebc1b47d,194df06f-a2a2-42bb-939b-331d0281a3af,DD_Doc_Details-ListView,1,"{""LINK"":""a95caaa0-a674-4f31-84d4-cbb52f0317f1""...",2023-04-09 22:49:15.633,02d08055-5bb7-4c85-926d-87d628637c25,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03
9,64b18dc9-69d0-4286-887f-7469ebc1b47d,02d08055-5bb7-4c85-926d-87d628637c25,DD_Docs_Out-ListView,1,"{""LINK"":""b1c13924-3f99-4e6c-a02a-34472f2c5320""...",2023-04-09 22:49:16.377,02d08055-5bb7-4c85-926d-87d628637c25,True,a95caaa0-a674-4f31-84d4-cbb52f0317f1,True,d4c614dd-3114-4326-8ddd-bb9d74382b03


In [829]:
# df.to_excel('outputs/Group_Fixed.xlsx')

In [830]:
# берем все записи где надо взять ID из Group_ID, если это операция апдейта то возвращаем значения из Group_ID в Group_ID_Fixed
Group_To_Fix = (df[(df['Group_ID'] != df['Group_ID_Fixed']) & (df['N_Operation'] == 0) & (df['C_Table'] == 'DD_Docs_Out-ListView')])['Group_ID'].unique()


In [831]:
df.loc[df['Group_ID'].isin(Group_To_Fix), 'Group_ID_Fixed'] = df['Group_ID']

In [832]:
df['Group_ID'] = df['Group_ID_Fixed']
df = df.drop(['Group_ID_Fixed'], axis=1)


## Заполняем JSON
Берем Group_ID и F_Doc_Details, заполняем значения

In [833]:
errors = 0

for record_id in df.index:
    record = df.loc[record_id]
    if (record['Use']) & (record['Fixed']):
        if record['C_Table'] == 'DD_Docs_Out-ListView':
            obj = json.loads(record['C_Json'])
            obj['LINK'] = record['Group_ID']
            obj['F_Doc_Details'] = {'LINK': record['F_Doc_Details']}
            df.loc[record_id, 'C_Json'] = json.dumps(obj)
            # df.loc[record_id, 'Fixed'] = True

        try:
            if record['C_Table'] == 'DD_Files-ListView':
                obj = json.loads(record['C_Json'])
                obj['F_Doc_Details'] = {'LINK': record['F_Doc_Details']}
                obj['F_Docs_Out'] = {'LINK': record['Group_ID']}
                df.loc[record_id, 'C_Json'] = json.dumps(obj)
                # df.loc[record_id, 'Fixed'] = True
            
        except TypeError as e:
            errors += 1
            df.loc[record_id, 'Fixed'] = False
            # print(record['Group_ID'], record['C_Record_ID'])

In [834]:
# df.to_excel('outputs/DD_Files.xlsx')

In [835]:
df[df['C_Record_ID'].isin(correct_records)].sort_values(by=['D_Date_Created'], ascending=True)

,F_User,C_Record_ID,C_Table,N_Operation,C_Json,D_Date_Created,Group_ID,Use,F_Doc_Details,Fixed
533,64b18dc9-69d0-4286-887f-7469ebc1b47d,684df633-d275-407b-8630-51d18cd11300,DD_Doc_Details-ListView,1,"{""LINK"":""8c110804-0515-46ac-bc75-e1677d67ea45""...",2023-04-09 18:42:55.083,8829a30f-1e54-4484-8d88-d14a53aaa17f,True,8c110804-0515-46ac-bc75-e1677d67ea45,True
534,64b18dc9-69d0-4286-887f-7469ebc1b47d,8829a30f-1e54-4484-8d88-d14a53aaa17f,DD_Docs_Out-ListView,1,"{""LINK"": ""8829a30f-1e54-4484-8d88-d14a53aaa17f...",2023-04-09 18:42:55.847,8829a30f-1e54-4484-8d88-d14a53aaa17f,True,8c110804-0515-46ac-bc75-e1677d67ea45,True
535,64b18dc9-69d0-4286-887f-7469ebc1b47d,51eeeb47-96e1-4479-9bc7-0d3e99d62b36,DD_Docs-ListView,1,"{""LINK"":""382f7b07-9e6f-42fb-8a81-05025b6227a0""...",2023-04-09 18:42:56.330,8829a30f-1e54-4484-8d88-d14a53aaa17f,True,8c110804-0515-46ac-bc75-e1677d67ea45,True
539,64b18dc9-69d0-4286-887f-7469ebc1b47d,a0c82551-eb25-41cd-9ddf-1993c6de5a2d,DD_Doc_Details-ListView,1,"{""LINK"":""253a53bb-262e-4495-8f82-a93aa36e6f10""...",2023-04-09 18:44:12.183,a23773d3-13eb-450f-a2e4-59ff1570cf4c,True,253a53bb-262e-4495-8f82-a93aa36e6f10,True
540,64b18dc9-69d0-4286-887f-7469ebc1b47d,a23773d3-13eb-450f-a2e4-59ff1570cf4c,DD_Docs_Out-ListView,1,"{""LINK"": ""a23773d3-13eb-450f-a2e4-59ff1570cf4c...",2023-04-09 18:44:13.007,a23773d3-13eb-450f-a2e4-59ff1570cf4c,True,253a53bb-262e-4495-8f82-a93aa36e6f10,True
...,...,...,...,...,...,...,...,...,...,...
63750,2e93ec41-0009-488f-a7d0-c2215edb590d,f1a8381e-b72e-4003-a528-0be23277c4b2,DD_Doc_Details-ListView,1,"{""LINK"":""c80ec678-ad9f-42e3-a49e-6fea91623bbb""...",2023-04-12 23:53:03.820,653adfea-8981-4945-987f-e93a68c2df27,True,c80ec678-ad9f-42e3-a49e-6fea91623bbb,True
63752,2e93ec41-0009-488f-a7d0-c2215edb590d,653adfea-8981-4945-987f-e93a68c2df27,DD_Docs_Out-ListView,0,"{""LINK"": ""653adfea-8981-4945-987f-e93a68c2df27...",2023-04-12 23:53:05.897,653adfea-8981-4945-987f-e93a68c2df27,True,c80ec678-ad9f-42e3-a49e-6fea91623bbb,True
63747,2e93ec41-0009-488f-a7d0-c2215edb590d,68eb1bf6-64da-4312-9cbf-9d78a8cc8308,DD_Files-ListView,0,"{""LINK"": ""bd98a8d8-30a0-fa77-1c48-6df368e1af25...",2023-04-12 23:53:07.007,653adfea-8981-4945-987f-e93a68c2df27,True,c80ec678-ad9f-42e3-a49e-6fea91623bbb,True
63749,2e93ec41-0009-488f-a7d0-c2215edb590d,9dea2903-0aa6-47bb-943c-8180e6468ac3,DD_Files-ListView,0,"{""LINK"": ""fdcc72ec-b53d-fa09-a8a3-d4051c9e5197...",2023-04-12 23:53:07.527,653adfea-8981-4945-987f-e93a68c2df27,True,c80ec678-ad9f-42e3-a49e-6fea91623bbb,True


In [840]:
# df[(df['Use'] == True) & (df['Fixed'] == True)].rename(columns={'Fixed':'is_fixed'}).to_excel('outputs/Export.xlsx')

df[~df['C_Record_ID'].isin(correct_records)].rename(columns={'Fixed':'is_fixed'}).to_excel('outputs/Export.xlsx')

## Правка типов актов
Пробуем поправить типы актов в зависимости от фотографий и типа маршрутного листа

In [847]:
df = df[~df['C_Record_ID'].isin(correct_records)]

In [884]:
type_errors = 0

df['Type_Fixed'] = ''
df['F_Types'] = ''
df['F_Categories_MRU'] = ''
df['DD_Files'] = ''

df['Details_C_Number'] = ''

groups = list(df['Group_ID'].unique())
for groups_id in groups:
    data = df[(df['Group_ID'] == groups_id) & (record['Use'] == True)]

    # вытащим типы маршрутов
    dd_docs_types = []
    dd_docs_categories = []

    dd_docs_index = data['C_Table'].isin(['DD_Docs-ListView'])
    dd_docs_data = data[dd_docs_index].apply(lambda x: json.loads(x['C_Json']), axis=1)
    
    if len(dd_docs_data) > 0:
        for i in dd_docs_data:
            if bool(i['F_Types']):
                dd_docs_types.append(i['F_Types']['LINK'])
            if bool(i['F_Categories_MRU']):
                dd_docs_categories.append(i['F_Categories_MRU']['LINK'])

    found_dd_docs_type = list(set(dd_docs_types))
    found_dd_docs_category = list(set(dd_docs_categories))

    if len(found_dd_docs_type) == 1:
        df.loc[data.index, 'F_Types'] = found_dd_docs_type[0]

    if len(found_dd_docs_category) == 1:
        df.loc[data.index, 'F_Categories_MRU'] = found_dd_docs_category[0]

    # вытащим типы файлов
    try:
        file_types = []

        dd_files_index = data['C_Table'].isin(['DD_Files-ListView'])
        dd_files_data = data[dd_files_index].apply(lambda x: json.loads(x['C_Json']), axis=1)
        
        if len(dd_files_data) > 0:
            for i in dd_files_data:
                if bool(i['F_Type']):
                    file_types.append(i['F_Type']['C_Const'])

        found_dd_files_type = list(set(file_types))

        if len(found_dd_files_type) > 0:
            df.loc[data.index, 'DD_Files'] = json.dumps(found_dd_files_type)
        
        if len(found_dd_files_type) == 0:
            df.loc[data.index, 'DD_Files'] = '[]'


    except TypeError as e:
        type_errors += 1
    

    # вытащим номера ПУ
    dd_doc_details_number = []

    dd_doc_details_index = data['C_Table'].isin(['DD_Doc_Details-ListView'])
    dd_doc_details_data = data[dd_doc_details_index].apply(lambda x: json.loads(x['C_Json']), axis=1)
    
    if len(dd_doc_details_data) > 0:
        for i in dd_doc_details_data:
            if bool(i['C_Number']):
                dd_doc_details_number.append(i['C_Number'])

    found_dd_doc_details_number = list(set(dd_doc_details_number))

    if len(found_dd_doc_details_number) == 1:
        df.loc[data.index, 'Details_C_Number'] = found_dd_doc_details_number[0]
    
    if len(found_dd_files_type) == 0:
        df.loc[data.index, 'Details_C_Number'] = ''
        
print(type_errors)

C:\Users\k-avtonomov.COMPULINK\AppData\Local\Temp\ipykernel_4452\2293895494.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Type_Fixed'] = ''
C:\Users\k-avtonomov.COMPULINK\AppData\Local\Temp\ipykernel_4452\2293895494.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['F_Types'] = ''
C:\Users\k-avtonomov.COMPULINK\AppData\Local\Temp\ipykernel_4452\2293895494.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] =

120


In [888]:
def F_Categories_MRU_N_Code(F_Categories_MRU):
    obj = {
        616890: '1',
        616892: '3',
        616891: '2',
        616894: '5',
        616895: '6',
        617425: '7',
        616893: '4',
        615001: '3'
    }

    return obj[F_Categories_MRU]


df['F_Categories_MRU'] = df['F_Categories_MRU'].apply(F_Categories_MRU_N_Code)

C:\Users\k-avtonomov.COMPULINK\AppData\Local\Temp\ipykernel_4452\1574990651.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['F_Categories_MRU'] = df['F_Categories_MRU'].apply(F_Categories_MRU_N_Code)


In [928]:
ds_types = [{ 
    'LINK': 2003,
    'C_Short_Name': 'АВЭ',
    'C_Name': 'Акт на возобновление электроснабжения', 
    'C_Const': 'DT_CONNECT',
    'Route_Types': ['20'],
    'Route_Categories': ['6'],
    'DS_File_Types': ['TF_DISCONNECT_PLACE']
},
{ 
    'LINK': 2002,
    'C_Short_Name': 'AОЭ',
    'C_Name': 'Акт на ограничение электроснабжения', 
    'C_Const': 'DT_DISCONNECT',
    'Route_Types': ['20'],
    'Route_Categories': ['5'],
    'DS_File_Types': []
},
{ 
    'LINK': 6,
    'C_Short_Name': 'АБУП',
    'C_Name': 'Акт о безучетном потреблении', 
    'C_Const': 'DT_NOT_CONTROL',
    'Route_Types': ['20'],
    'Route_Categories': ['1', '2'],
    'DS_File_Types': [
        'TF_ACT',
        'TF_DEVICE'
    ]
},
{ 
    'LINK': 2009,
    'C_Short_Name': 'АППУ',
    'C_Name': 'Акт проверки ПУ', 
    'C_Const': 'DT_VERIFICATION_CHECK',
    'Route_Types': ['20'],
    'Route_Categories': ['7'],
    'DS_File_Types': ['TF_ACT', 'IVP_DEVICE_BEFORE_VERIFICATION']
},
{ 
    'LINK': 4,
    'C_Short_Name': 'АИП',
    'C_Name': 'Акт проверки узла учета', 
    'C_Const': 'DT_CONTROL_CHECK',
    'Route_Types': ['1', '20'],
    'Route_Categories': ['1', '2'],
    'DS_File_Types': [
        'TF_DEVICE'
    ]
},
{ 
    # 0 фото
    'LINK': 2004,
    'C_Short_Name': 'ОЗО',
    'C_Name': 'Отмена заявки на ограничение', 
    'C_Const': 'DT_CANCEL_DISCONNECT',
    'Route_Types': ['20'],
    'Route_Categories': ['5'],
    'DS_File_Types': []
},
{ 
    'LINK': 8,
    'C_Short_Name': 'АОВ',
    'C_Name': 'Акт недопуска', 
    'C_Const': 'DT_FAILURE',
    'Route_Types': ['1', '20'],
    'Route_Categories': ['1', '2', '3', '4', '5'],
    'DS_File_Types': [
        'TF_PHOTO'
    ]
},
{ 
    'LINK': 3,
    'C_Short_Name': 'АСКП',
    'C_Name': 'Акт снятия контрольного показания', 
    'C_Const': 'DT_CONTROL_METER_READINGS',
    'Route_Types': ['1', '20'],
    'Route_Categories': ['1', '7'],
    'DS_File_Types': [
        'TF_DEVICE'
    ]
},
{ 
    'LINK': 2001,
    'C_Short_Name': 'АЗПУ',
    'C_Name': 'Акт замены ПУ', 
    'C_Const': 'DT_REPLACEMENT',
    'Route_Types': ['20'],
    'Route_Categories': ['3'],
    'DS_File_Types': [
        'TF_ACT',
        'TF_SEAL'
    ]
},
{ 
    'LINK': 2006,
    'C_Short_Name': 'АДПУТП',
    'C_Name': 'Акт допуска ПУ по ТП', 
    'C_Const': 'DT_ADMISSION_DEVICE_TP',
    'Route_Types': ['20'],
    'Route_Categories': ['4'],
    'DS_File_Types': [
        'TF_DEVICE',
        'TF_RATE_INDICATION',
        'TF_SEAL'
    ]
}]

In [917]:
df['D_Date_Created'].apply(lambda x: '{:%d%m%y}'.format(x))

0        090423
1        090423
2        090423
3        090423
4        090423
          ...  
74955    120423
74956    120423
74957    120423
74958    120423
74959    120423
Name: D_Date_Created, Length: 32713, dtype: object

In [938]:
index = df[df['C_Table'] == 'DD_Docs_Out-ListView'].index

for record_id in index:
    record = df.loc[record_id]
    if (record['Use']) & (record['Fixed']):
        F_Types	= record['F_Types']
        F_Categories_MRU = record['F_Categories_MRU']
        DD_Files = record['DD_Files']

        selected_type = None
        for d_type in ds_types:
            
            # print(d_type)
            if len(list(set(set(d_type['Route_Categories']) & set(F_Categories_MRU)))) == 0:
                continue

            if len(list(set(set(d_type['Route_Types']) & set([str(F_Types)])))) == 0:    
                continue
            
            try:
                _DD_Files = json.loads(DD_Files)
            except json.JSONDecodeError:
                _DD_Files = []

            DS_File_Types = list(set(set(d_type['DS_File_Types']) & set(_DD_Files)))

            if len(DS_File_Types) == 0:
                if d_type['C_Const'] != 'DT_CANCEL_DISCONNECT':
                    continue
                
                
            elif (len(DD_Files) >= len(DS_File_Types)):
                selected_type = d_type

        # print(selected_type)

        obj = json.loads(record['C_Json'])

        if selected_type != None:
            date = '{:%d%m%y}'.format(record['D_Date_Created'])
            
            obj['C_Doc_Number'] = f"{selected_type['C_Short_Name']}-{record['Details_C_Number']}-{date}"
            
            df.loc[record_id, 'C_Name_Previous'] = obj['F_Types']['C_Name']
            
            obj['F_Types'] = {
                'LINK': selected_type['LINK'],
                'C_Name': selected_type['C_Name'],
                'C_Const': selected_type['C_Const']
            }
            
            df.loc[record_id, 'C_Name_Fixed'] = selected_type['C_Name']
            df.loc[record_id, 'C_Json_Fixed'] = json.dumps(obj)
        # print(obj)
        # if record['C_Table'] == 'DD_Docs_Out-ListView':
        #     obj = json.loads(record['C_Json'])
        #     obj['LINK'] = record['Group_ID']
        #     obj['F_Doc_Details'] = {'LINK': record['F_Doc_Details']}
        #     df.loc[record_id, 'C_Json'] = json.dumps(obj)
        #     # df.loc[record_id, 'Fixed'] = True

        # try:
        #     if record['C_Table'] == 'DD_Files-ListView':
        #         obj = json.loads(record['C_Json'])
        #         obj['F_Doc_Details'] = {'LINK': record['F_Doc_Details']}
        #         obj['F_Docs_Out'] = {'LINK': record['Group_ID']}
        #         df.loc[record_id, 'C_Json'] = json.dumps(obj)
        #         # df.loc[record_id, 'Fixed'] = True
            
        # except TypeError as e:
        #     errors += 1
        #     df.loc[record_id, 'Fixed'] = False
            # print(record['Group_ID'], record['C_Record_ID'])

C:\Users\k-avtonomov.COMPULINK\AppData\Local\Temp\ipykernel_4452\278969987.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[record_id, 'C_Name_Previous'] = obj['F_Types']['C_Name']


In [951]:
df[(~df['C_Name_Fixed'].isna()) & (df['C_Name_Fixed'] != df['C_Name_Previous'])]

,F_User,C_Record_ID,C_Table,N_Operation,C_Json,D_Date_Created,Group_ID,Use,F_Doc_Details,Fixed,Type_Fixed,F_Types,F_Categories_MRU,DD_Files,Details_C_Number,C_Json_Fixed,C_Name_Fixed,C_Name_Previous
2424,cd5c7060-4649-4a52-84ec-607a686a44b7,f5da6921-aae6-4749-b7ed-076e4eca1c45,DD_Docs_Out-ListView,0,"{""LINK"": ""f5da6921-aae6-4749-b7ed-076e4eca1c45...",2023-04-10 09:25:06.177,f5da6921-aae6-4749-b7ed-076e4eca1c45,True,3e96bfa8-30dd-4769-8bcd-b3d7876c9d98,True,,20,1,"[""TF_DEVICE"", ""TF_ACT""]",316766,"{""LINK"": ""f5da6921-aae6-4749-b7ed-076e4eca1c45...",Акт снятия контрольного показания,Акт проверки узла учета
2432,cd5c7060-4649-4a52-84ec-607a686a44b7,52f2a918-3e8c-48c2-b02b-b1c0ba15e0ed,DD_Docs_Out-ListView,1,"{""LINK"": ""f5da6921-aae6-4749-b7ed-076e4eca1c45...",2023-04-10 09:26:32.537,f5da6921-aae6-4749-b7ed-076e4eca1c45,True,3e96bfa8-30dd-4769-8bcd-b3d7876c9d98,True,,20,1,"[""TF_DEVICE"", ""TF_ACT""]",316766,"{""LINK"": ""f5da6921-aae6-4749-b7ed-076e4eca1c45...",Акт снятия контрольного показания,Акт проверки узла учета
2640,cd5c7060-4649-4a52-84ec-607a686a44b7,ba117cdd-92a8-4699-ae58-8f86739450b3,DD_Docs_Out-ListView,0,"{""LINK"": ""ba117cdd-92a8-4699-ae58-8f86739450b3...",2023-04-10 09:30:40.003,ba117cdd-92a8-4699-ae58-8f86739450b3,True,43bc272c-2cc1-44dd-b6ab-17ba175c17f9,True,,20,1,"[""TF_DEVICE"", ""TF_ACT""]",835126,"{""LINK"": ""ba117cdd-92a8-4699-ae58-8f86739450b3...",Акт снятия контрольного показания,Акт проверки узла учета
2805,cd5c7060-4649-4a52-84ec-607a686a44b7,acda8a50-4a48-4375-93d4-dda137a65c7a,DD_Docs_Out-ListView,0,"{""LINK"": ""acda8a50-4a48-4375-93d4-dda137a65c7a...",2023-04-10 09:33:59.187,acda8a50-4a48-4375-93d4-dda137a65c7a,True,28f821d5-c72e-4a2c-bfd2-bf3799c74592,True,,20,1,"[""TF_DEVICE"", ""TF_ACT""]",391601,"{""LINK"": ""acda8a50-4a48-4375-93d4-dda137a65c7a...",Акт снятия контрольного показания,Акт проверки узла учета
2814,cd5c7060-4649-4a52-84ec-607a686a44b7,75489299-57fb-4619-97f5-316b61cba265,DD_Docs_Out-ListView,0,"{""LINK"": ""75489299-57fb-4619-97f5-316b61cba265...",2023-04-10 09:57:35.620,75489299-57fb-4619-97f5-316b61cba265,True,28f821d5-c72e-4a2c-bfd2-bf3799c74592,True,,20,1,"[""TF_DEVICE"", ""TF_ACT""]",391601,"{""LINK"": ""75489299-57fb-4619-97f5-316b61cba265...",Акт снятия контрольного показания,Акт проверки узла учета
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53614,acab0877-8538-46c5-91f4-370268631935,4a3ae075-e756-4b17-b3c6-c5f397b1d4e5,DD_Docs_Out-ListView,0,"{""LINK"": ""4a3ae075-e756-4b17-b3c6-c5f397b1d4e5...",2023-04-11 11:32:33.417,4a3ae075-e756-4b17-b3c6-c5f397b1d4e5,True,8f387adf-3653-4883-92f9-700fa0595c18,True,,20,1,"[""TF_PHOTO""]",23610381,"{""LINK"": ""4a3ae075-e756-4b17-b3c6-c5f397b1d4e5...",Акт недопуска,Акт снятия контрольного показания
54345,f19c0102-7b28-477a-bf69-0c4cd19b3cc1,46ace9c5-cb88-4bc1-83ab-1d42661179ba,DD_Docs_Out-ListView,0,"{""LINK"": ""46ace9c5-cb88-4bc1-83ab-1d42661179ba...",2023-04-11 11:27:12.497,46ace9c5-cb88-4bc1-83ab-1d42661179ba,True,70cfecbb-bdcb-48d1-9445-3efa2c4ef566,True,,20,1,"[""TF_DEVICE"", ""TF_ACT""]",36639513,"{""LINK"": ""46ace9c5-cb88-4bc1-83ab-1d42661179ba...",Акт снятия контрольного показания,Акт проверки узла учета
54883,c087b63b-fc0c-4215-b004-4495a3912ec9,42db045a-d73a-4370-865b-691279a45312,DD_Docs_Out-ListView,0,"{""LINK"": ""42db045a-d73a-4370-865b-691279a45312...",2023-04-11 11:29:37.423,42db045a-d73a-4370-865b-691279a45312,True,04231d70-5052-4ea8-9abb-28d81e2ea362,True,,20,1,"[""TF_PHOTO""]",3190249652351,"{""LINK"": ""42db045a-d73a-4370-865b-691279a45312...",Акт недопуска,Акт снятия контрольного показания
56128,36016f01-45e3-4699-a1c5-369c04970ef7,f938f567-d155-411a-b059-3d81aa60a038,DD_Docs_Out-ListView,0,"{""LINK"": ""f938f567-d155-411a-b059-3d81aa60a038...",2023-04-11 17:16:31.597,f938f567-d155-411a-b059-3d81aa60a038,True,61761ab1-ea8d-453e-b133-ad2c424ab507,True,,20,2,"[""TF_DEVICE"", ""TF_ACT""]",0610120163,"{""LINK"": ""f938f567-d155-411a-b059-3d81aa60a038...",Акт проверки узла уче

In [ ]:
# df = df[(df['Use'] == True) & (df['Fixed'] == True)]

In [841]:
# df['DD_Docs_Type'] = ''

# groups = list(df['Group_ID'].unique())
# for groups_id in groups:
#     data = df[(df['Group_ID'] == groups_id) & (record['Use'] == True)]

#     dd_docs_types = []
#     dd_docs_categories = []

#     dd_docs_index = data['C_Table'].isin(['DD_Docs-ListView'])
#     dd_docs_data = data[meters_index].apply(lambda x: json.loads(x['C_Json']), axis=1)
#     if len(dd_docs_data) > 0:
#         for i in dd_docs_data:
#             if bool(i['F_Type']):
#                 dd_docs_types.append(i['F_Type']['LINK'])
#             if bool(i['F_Categories_MRU']):
#                 dd_docs_categories.append(i['F_Categories_MRU']['LINK'])

#     found_dd_docs_type = list(set(dd_docs_types))
#     found_dd_docs_category = list(set(dd_docs_categories))
    
#     if len(found_dd_docs_type) == 1:
#         df.loc[data.index, 'DD_Docs_Type'] = found_dd_docs_type
        
#     if len(found_dd_docs_category) == 1:
#         df.loc[data.index, 'DD_Docs_Category'] = found_dd_docs_category
    
#     # details_index = data['C_Table'].isin(['DD_Doc_Details-ListView'])
#     # details_data = data[details_index].apply(lambda x: json.loads(x['C_Json']), axis=1)
#     # if len(details_data) > 0:
#     #     for i in details_data:
#     #         detail_links.append(i['LINK'])

#     # found_detail = list(set(detail_links))

#     # if len(found_detail) > 1:
#     #     df.loc[data.index, 'F_Doc_Details'] = json.dumps(found_detail)
#     #     df.loc[data.index, 'Fixed'] = False

#     # if len(found_detail) == 1:
#     #     df.loc[data.index, 'F_Doc_Details'] = found_detail[0]
#     #     df.loc[data.index, 'Fixed'] = True

#     # if len(found_detail) == 0:
#     #     df.loc[data.index, 'F_Doc_Details'] = ''
